In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

/home/nsml/.conda/envs/fc_peft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting ds_accelerator to cuda (auto detect)


In [2]:
model_path = "./ckpts/my_chatgpt_3.8b/checkpoint-31671"
config = PeftConfig.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [3]:
inference_model = PeftModel.from_pretrained(model, model_path)
inference_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 3072)
        (layers): ModuleList(
          (0-31): 32 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=3072, out_features=9216, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=9216, bias=False)
                )
     

In [4]:
pipe = pipeline(
    "text-generation",
    model=inference_model,
    tokenizer=tokenizer,
    device=0,
    torch_dtype=torch.bfloat16
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

In [5]:
def build_prompt(user_input):
    return f"### Human\n{user_input}\n\n### Bot\n"

In [8]:
def ask(text):
    result = pipe(build_prompt(text),
                   return_full_text=False,
                   do_sample=False,
                   max_new_tokens=256)
    return result[0]['generated_text']

In [9]:
print(ask("토니 로빈스가 설명하는 인간의 6가지 욕구를 요약해 보세요..."))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. 안전: 안전은 인간의 기본 욕구 중 하나로, 안전하지 않은 상황에서는 불안과 스트레스를 느끼게 됩니다. 안전은 건강, 교육, 직업, 재정적 안정 등 다양한 영역에서 중요한 역할을 합니다.2. 소속감: 소속감은 인간의 기본 욕구 중 하나로, 다른 사람들과 함께 있을 때 안정감을 느끼고 자신의 가치와 정체성을 확인할 수 있습니다. 소속감은 가족, 친구, 공동체와 같은 사회적 관계에서 중요한 역할을 합니다.3. 자기표현: 자기표현은 인간의 기본 욕구 중 하나로, 자신의 감정과 생각을 표현하고 다른 사람들과 소통하는 것을 의미합니다. 자기표현은 개인의 정체성, 자존감, 자기 존중감을 형성하는 데 중요한 역할을 합니다.4. 즐거움: 즐거움은 인간의 기본 욕구 중 하나로, 삶의 모든 측면에서 즐거움을 느끼는 것을 의미합니다. 즐거움은 스포츠, 예술, 자연과 문화를 즐기는 것과 같은 다양한 영역에서 중요한 역할을 합니다.5. 의미: 의미는 인간의 기본 욕구 중 하나로, 삶의 목적과 의미를 찾는 것을 의미합니다


# Generation HyperParameter Tuning

In [21]:
def ask(text):
    result = pipe(build_prompt(text),
                   return_full_text=False,
                   do_sample=True,
                   top_p=0.3,
                   max_new_tokens=128,
                   repetition_penalty=1.2)
    return result[0]['generated_text'].split("\n\n")[0]

In [13]:
print(ask("토니 로빈스가 설명하는 인간의 6가지 욕구를 요약해 보세요..."))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. 안전: 안전은 삶에서 가장 중요한 요소 중 하나입니다. 우리 모두는 위험을 피하고, 위협에 대처할 수 있으며, 다른 사람들과 연결되어 있다는 느낌이 필요합니다.2. 자기계발: 자기 계발은 자신에게 투자하여 개인적 성장과 발전을 추구하는 것으로 정의됩니다. 여기에는 독서와 글쓰기 등 다양한 기술 습득 및 개발 활동이 포함될 수 있습니다.3. 관계: 관계는 사랑과 신뢰로 특징지어지며 모든 인간관계의 기초이자 핵심입니다. 우리는 가족이나 친구 또는 직장 동료 사이에서 서로


# Prompt Engineering

## 프롬프트 엔지니어링 팁
- 요구 조건은 쉽고 간결하게
- 필요한 요구 조건들을 잘 나열
- 구체적으로 명시
- N-shot 예제를 함께 주면서

# 광고 문구 생성

In [14]:
sample_1 = "나이키 신발 광고 문구를 작성해줘"

In [16]:
print(ask(sample_1))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"당신의 발을 위한 최고의 선택, 나이키 에어 줌 페가수스 83"


### 필요조건 나열, 구체적으로 명시

In [17]:
sample_2 = """다음의 요구 사항에 따라 광고 문구를 작성해
제품명: 나이키 신발
설명: 착화감이 좋고 편안한
"""

In [18]:
print(ask(sample_2))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


나이키 신발은 발을 감싸는 편안함과 내구성으로 유명합니다. 이 제품은 발의 피로와 불편함을 최소화하기 위해 설계되었습니다. 또한, 이 제품은 뛰어난 통기성을 제공하여 발에서 발생할 수 있는 땀이나 습기가 빠르게 배출됩니다. 이 신발은 일상 생활용 및 운동용 모두 적합합니다.


In [19]:
sample_3 = """다음의 요구 사항에 따라 광고 문구를 작성해
제품명: 나이키 신발
설명: 착화감이 좋고 편안한
톤앤매너: 유쾌하고, 발랄하게
"""

In [22]:
print(ask(sample_3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"나이키 신발을 신으면 자신감과 에너지가 넘쳐납니다. 이 운동화는 여러분에게 완벽한 스타일을 선사합니다."


In [23]:
sample = """다음의 요구 사항에 따라 광고 문구를 작성해
제품명: 겨울 패딩
설명: 바람을 잘막아주는, 오래 입는
톤앤매너: 참신하게
"""

In [24]:
print(ask(sample))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


겨울 패딩으로 따뜻하고 편안한 여행을 즐겨보세요. 가볍고 방수성이 뛰어난 이 제품은 여러분이 어디로 가든지 당신과 함께할 것입니다. 바람막이와 방수 기능이 있어 비가 오거나 눈이 내리더라도 걱정 없이 야외 활동을 즐길 수 있습니다. 또한 이 제품은 내구성이 뛰어나서 오랫동안 사용해도 처음처럼 튼튼합니다. 이제 여러분만의 개성 넘치는 스타일을 연출하여 겨울에도 멋지게 보내시길 바랍니다!


## N-shot 예제를 함께 주면서

In [25]:
fewshot = """\
### User
다음의 요구 사항에 따라 광고 문구를 작성해
제품명: 나이키 신발
설명: 착화감이 좋고 편안한
톤앤매너: 유쾌하고, 발랄하게

### Bot
- 너무 편안해서 때문에 발이 녹아버릴 것 같아요! 나이키 신발 착화감 짱짱!
- 나이키 신발 착화감이 너무 좋고 편안해서 즐거워요 :-)
- 신은 것 같지 않은 편안함! 나이키 신발 강추합니다!

### User
다음의 요구 사항에 따라 광고 문구를 작성해
제품명: 겨울 패딩
설명: 바람을 잘막아주는, 오래 입는
톤앤매너: 참신하게

### Bot
- 총알같은 바람도 막아내는 겨울패딩
- 바람을 잘 막아주고, 오래 읿을 수 있는 나의 겨울 친구 겨울 패딩
- 겨울이 왔으면 좋겠어요. 이 겨울 패딩하나면 겨울이 하나도 안무서워요! 저랑 같이 오래입어요"""

In [26]:
def build_prompt(user_input):
    return f"{fewshot}\n\n### User\n{user_input}\n\n### Bot\n"

In [32]:
def ask(text):
    result = pipe(build_prompt(text),
                   return_full_text=False,
                   do_sample=True,
                   top_p=0.3,
                   max_new_tokens=128,
                   repetition_penalty=1.3)
    return result[0]['generated_text'].split("\n\n")[0]

In [33]:
sample = """다음의 요구 사항에 따라 광고 문구를 작성해
제품명: 아이스아메리카노
설명: 시지않고, 고소한
톤앤매너: 고급스럽게
"""
print(ask(sample))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


- 시원하고 깔끔한 맛으로 유명한 커피 한잔 하실래요?
- 얼음 동동 띄운 아메리카노 한 잔이면 하루가 상쾌해져요!
- 차가운 음료와 함께라면 더 맛있어지죠! 이제부터 시작입니다.
